In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!source /content/drive/MyDrive/colab_env/bin/activate

In [ ]:
! pip install geemap wxee

In [19]:
import glob
import os
import re
import sys
import pandas as pd
import numpy as np
import xarray as xr
import ee
import wxee
import rioxarray
import rasterio
import pickle
import pyproj
import itertools
import pickle

In [20]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file="/content/drive/MyDrive/private-key.json"
)

ee.Initialize(credentials)

In [34]:
dts_data = pd.read_csv("/content/drive/MyDrive/EC_Tower/station_metadata_len.csv")
df = dts_data[:][:16]

In [35]:
# def get_soil_prop(param):
#     """
#     This function returns soil properties image
#     param (str): must be one of:
#         "ppt"      - Daily total precipitation (including rain and melted snow)
#         "tmean"    - Daily mean temperature (calculated as (tmin+tmax)/2)
#         "tmin"     - Daily minimum temperature
#         "tmax"     - Daily maximum temperature
#         "tdmean"   - Daily mean dew point temperature
#         "vpdmin"   - Daily minimum vapor pressure deficit
#         "vpdmax"   - Daily maximum vapor pressure deficit

#     """
#     snippet = "OREGONSTATE/PRISM/AN81d"
#     if param == "ppt":  # ppt
#         # Define the scale factor in accordance with the dataset description.
#         scale_factor = 1 * 0.01

#     elif param == "clay":  # Clay fraction [%w]
#         snippet = "OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02"
#         # Define the scale factor in accordance with the dataset description.
#         scale_factor = 1 * 0.01

#     elif param == "orgc":  # Organic Carbon fraction [g/kg]
#         snippet = "OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02"
#         # Define the scale factor in accordance with the dataset description.
#         scale_factor = 5 * 0.001  # to get kg/kg
#     else:
#         return print("error")

#     # Apply the scale factor to the ee.Image.
#     dataset = ee.Image(snippet).multiply(scale_factor)

#     return dataset

crs = 4326
# Function to create an ee.Geometry.Point from a row
def create_ee_point(row):
    return ee.Geometry.Point([row['Longitude'], row['Latitude']])

# Apply the function to each row of the DataFrame
ee_points = df.apply(create_ee_point, axis=1)


# Define a point of interest
# point = ee.Geometry.Point([-100.55, 40.71])

data = []

for point in ee_points:
  dataset = ee.ImageCollection('OREGONSTATE/PRISM/AN81d').filter(ee.Filter.date('2009-01-01', '2020-12-31')).filter(ee.Filter.bounds(point))
  dataset = dataset.set("system:time_start", ee.Date("2009-01-01"))
  arr_dataset = dataset.wx.to_xarray(scale=4638.3, crs=f'EPSG:{crs}')
  da_dataset = arr_dataset.rio.write_crs(f"epsg:{crs}", inplace=True)
  break
  # data.appnd(da_dataset)
  # merged_dataset = xr.merge([xds_repr_match__soil_era_evaporation, xds_repr_match__soil_era_transpiration1, xds_repr_match__soil_era_transpiration2, xds_repr_match__soil_era_transition])

da_dataset





Requesting data:   0%|          | 0/4382 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Specify the directory path and file pattern

file_path = '/mnt/e/flux_ET_dataset/daily_data_files/*.csv'
csv_files = glob.glob(file_path)


In [ ]:
cc = csv_files[20]
word_to_remove = cc[-15:]

# Use regular expression to replace the word with an empty string
pattern = re.escape(word_to_remove)
new_sentence = re.sub(pattern, "", cc[40:])
new_sentence

'LYS_NE'

In [ ]:
def find_station_name(file_path):

    word_to_remove = file_path[-15:]
    # Use regular expression to replace the word with an empty string
    pattern = re.escape(word_to_remove)
    station_name = re.sub(pattern, "", file_path[40:])
    return station_name

In [ ]:
dts_data = pd.read_csv(csv_files[0])
dts_data

,date,H,G,Rn,LE,Rn_subday_gaps,H_subday_gaps,LE_subday_gaps,G_subday_gaps,rso,...,ET,ET_corr,gridMET_ETr,gridMET_ETo,gridMET_prcp,EToF,EToF_filtered,ET_fill,ET_gap,ET_fill_val
0,2011-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162.406991,...,NaN,0.038096,1.9,1.4,0.0,NaN,0.027212,0.038096,True,0.038096
1,2011-11-16,19.330913,-4.123441,19.871779,1.787188,0.0,0.0,0.0,0.0,161.124982,...,0.062929,0.066505,3.1,2.0,0.0,0.033253,0.022236,0.044472,False,NaN
2,2011-11-17,23.638188,-4.530587,20.304799,1.168036,0.0,0.0,0.0,0.0,159.875084,...,0.041128,0.043253,1.8,1.3,0.0,0.033272,0.020285,0.026370,False,NaN
3,2011-11-18,13.948604,-0.035418,20.660817,0.861524,0.0,0.0,0.0,0.0,158.657738,...,0.030335,0.031731,3.2,2.1,0.0,0.015110,0.035948,0.075492,False,NaN
4,2011-11-19,15.151694,-7.812638,8.425509,0.416747,0.0,0.0,0.0,0.0,157.473372,...,0.014674,0.015349,3.3,2.1,0.0,0.007309,0.043339,0.091013,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2013-11-10,20.633071,-3.603290,19.143068,3.007674,0.0,0.0,0.0,0.0,169.282199,...,0.105903,0.104215,3.1,1.9,0.0,0.054850,0.027568,0.052380,False,NaN
727,2013-11-11,24.509524,-3.292027,21.714179,1.839492,0.0,0.0,0.0,0.0,167.846780,...,0.064770,0.063600,3.8,2.3,0.0,0.027652,0.027539,0.063339,False,NaN
728,2013-11-12,9.491723,2.988097,10.790397,1.056688,0.0,0.0,0.0,0.0,166.441042,...,0.037207,0.036535,4.5,2.8,0.0,0.013048,0.028163,0.078858,False,NaN
729,2013-11-13,15.738241,-6.585032,16.053082,2.217967,0.0,0.0,0.0,0.0,165.065500,...,0.078097,0.076686,3.6,2.2,0.0,0.034857,0.029536,0.064979,False,NaN


In [ ]:
dfs = []

for file in csv_files:

    ts_data = pd.read_csv(file)
    station_name = find_station_name(file)

    ts_data = ts_data.assign(station = station_name)

    dfs.append(ts_data)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df

,date,H,G,Rn,LE,Rn_subday_gaps,H_subday_gaps,LE_subday_gaps,G_subday_gaps,rso,...,theta_38,theta_40,theta_37,theta_33,theta_35,WD_1_6_1,WD_1_5_1,WS_1_6_1,WS_1_5_1,SW_IN_1_1_3
0,2011-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162.406991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-11-16,19.330913,-4.123441,19.871779,1.787188,0.0,0.0,0.0,0.0,161.124982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-11-17,23.638188,-4.530587,20.304799,1.168036,0.0,0.0,0.0,0.0,159.875084,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-11-18,13.948604,-0.035418,20.660817,0.861524,0.0,0.0,0.0,0.0,158.657738,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-11-19,15.151694,-7.812638,8.425509,0.416747,0.0,0.0,0.0,0.0,157.473372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349597,2007-08-27,63.690475,-1.939723,119.097462,NaN,0.0,0.0,13.0,0.0,313.726592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349598,2007-08-28,57.191733,-3.528373,104.172713,33.437424,0.0,0.0,0.0,0.0,311.883288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349599,2007-08-29,66.305009,0.979109,117.741026,27.717823,0.0,0.0,0.0,0.0,310.019390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349600,2007-08-30,51.968193,-2.942493,77.852393,19.561097,0.0,0.0,0.0,0.0,308.135324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
